In [1]:
from Environments.TradingEnv_ray import SineMarketEnv
from Environments.TradingEnv_ray import Env_ray

import numpy as np
import matplotlib.pyplot as plt
import ray
import gym
from ray.tune.registry import register_env
from ray.rllib.agents.ppo.ddppo import DEFAULT_CONFIG
from Agents.C_DDPPO import C_DDPPOTrainer
import os
import gc
gc.collect()

80

In [2]:
try:
    ray.init(num_gpus = 4, num_cpus = 4)
except:
    ray.shutdown()
    ray.init(num_gpus = 4, num_cpus = 4)

2020-09-26 16:25:37,171	INFO resource_spec.py:231 -- Starting Ray with 4.15 GiB memory available for workers and up to 2.1 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-26 16:25:37,679	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


In [3]:
trainer_config = DEFAULT_CONFIG.copy()

In [4]:
trainer_config['model']

{'conv_filters': None,
 'conv_activation': 'relu',
 'fcnet_activation': 'tanh',
 'fcnet_hiddens': [256, 256],
 'free_log_std': False,
 'no_final_linear': False,
 'vf_share_layers': True,
 'use_lstm': False,
 'max_seq_len': 20,
 'lstm_cell_size': 256,
 'lstm_use_prev_action_reward': False,
 'state_shape': None,
 'framestack': True,
 'dim': 84,
 'grayscale': False,
 'zero_mean': True,
 'custom_model': None,
 'custom_model_config': {},
 'custom_action_dist': None,
 'custom_preprocessor': None,
 'custom_options': -1}

In [5]:
trainer_config

{'num_workers': 2,
 'num_envs_per_worker': 5,
 'rollout_fragment_length': 100,
 'sample_batch_size': -1,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': -1,
 'model': {'conv_filters': None,
  'conv_activation': 'relu',
  'fcnet_activation': 'tanh',
  'fcnet_hiddens': [256, 256],
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  'state_shape': None,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None,
  'custom_options': -1},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {},
 'env': None,
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preprocessor_pref': 'deepmind',
 'lr': 5e-05,
 'monitor'

In [6]:
trainer_config['model']['use_lstm'] = True
trainer_config['model']['lstm_cell_size'] = 512
trainer_config['num_gpus'] = 0
trainer_config['num_gpus_per_worker'] = 4
trainer_config['num_envs_per_worker'] = 2
trainer_config['gamma'] = 0
trainer_config['entropy_coeff'] = 0
trainer_config['framework'] = 'torch'
trainer_config['num_workers'] = 1
trainer_config['horizon'] = 1000
trainer_config['rollout_fragment_length'] = 1000
trainer_config['model']['framestack'] = False
trainer_config['model']['fcnet_hiddens'] = [1024, 1024]
trainer_config['env_config'] = {'last_states_num': None,
                               'action_mode' : 1,
                               'version' : 'long',
                               'curiosity_reward': 0}
#trainer_config['entropy_coeff_schedule'] = 0

In [7]:
trainer = C_DDPPOTrainer(trainer_config, Env_ray)

2020-09-26 16:25:39,871	ERROR syncer.py:46 -- Log sync requires rsync to be installed.
2020-09-26 16:25:39,872	INFO trainer.py:632 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=6283) /home/miner/RL/lib/python3.7/site-packages/ray/rllib/utils/torch_ops.py:149: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
(pid=6283)   tensor = torch.from_numpy(np.asarray(item))
2020-09-26 16:25:55,216	INFO trainable.py:251 -- Trainable.setup took 15.345 seconds. If your trainable is slow to initialize, consider setting reuse_act

In [8]:
best_reward = -np.inf
trainer.save()
hall_of_fame = [0]
last_checkpoint = 0

(pid=6283) [W ProcessGroupGloo.cpp:471] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


In [9]:
%%time
for i in range(50):
    print("Training iteration {}...".format(i))
    results = trainer.train()
    this_reward = results['episode_reward_max']
    avg_reward = results['episode_reward_mean']
    if this_reward > best_reward:
        best_reward = this_reward
        trainer.save()
        path = trainer.logdir + 'checkpoint_{0}/checkpoint-{0}'.format(last_checkpoint)
        os.remove(path)
        last_checkpoint = i + 1
        hall_of_fame.append(i+1)
        print('New best reward')
        print(best_reward)
    if i % 10 == 0:
        print('Best Reward So Far')
        print(best_reward)
        print(avg_reward)

Training iteration 0...


RayTaskError(ValueError): [36mray::RolloutWorker.par_iter_next()[39m (pid=6283, ip=192.168.1.103)
  File "python/ray/_raylet.pyx", line 479, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 432, in ray._raylet.execute_task.function_executor
  File "/home/miner/RL/lib/python3.7/site-packages/ray/util/iter.py", line 1156, in par_iter_next
    return next(self.local_it)
  File "/home/miner/RL/lib/python3.7/site-packages/ray/util/iter.py", line 785, in apply_foreach
    for item in it:
  File "/home/miner/RL/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 281, in gen_rollouts
    yield self.sample()
  File "/home/miner/RL/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 563, in sample
    batches = [self.input_reader.next()]
  File "/home/miner/RL/lib/python3.7/site-packages/ray/rllib/evaluation/sampler.py", line 71, in next
    batches = [self.get_data()]
  File "/home/miner/RL/lib/python3.7/site-packages/ray/rllib/eval

In [10]:
trainer.save()

'/home/miner/ray_results/DDPPO_Env_ray_2020-09-26_16-25-39jnlo3hdz/checkpoint_0/checkpoint-0'

In [11]:
best_reward

-inf

In [12]:
#path = trainer.logdir + 'checkpoint_{0}/checkpoint-{0}'.format(hall_of_fame[-1])

In [13]:
#trainer.load_checkpoint(path)

In [14]:
import pandas as pd

In [15]:
training = pd.read_csv(trainer.logdir + 'progress.csv')
plt.plot(training['episode_reward_mean'])

EmptyDataError: No columns to parse from file

In [ ]:
training.columns

In [ ]:
import torch

In [ ]:
env = Env_ray(env_config = {'last_states_num': None, 'action_mode': 1, 'version': 'long', 'curiosity_reward':0})
obs = env.reset()

done = False
cumulative_reward = 0
prices = []
assets = []
actions = []
states = [obs]
rewards = []
hidden = [torch.zeros(512),torch.zeros(512)]
infos = []
for i in range(999):
    action, hidden, info = trainer.compute_action(obs, hidden)
    obs, reward, done, results = env.step(action)
    cumulative_reward += reward
    rewards.append(reward)
    actions.append(action)
    assets.append(env.last_assets)
    prices.append(obs[0])
    states.append(obs)
    infos.append(info)
print("Cumulative reward you've received is: {}. Congratulations!".format(cumulative_reward))
print("Asset_Gain {}".format(assets[-1] -assets[0]))

In [ ]:
len(env.curiosities)

In [ ]:
sum(env.curiosities)/len(env.curiosities)

In [ ]:
sell100 = np.ma.masked_where(np.array(actions) != 6, prices)
buy10 = np.ma.masked_where(np.array(actions) != 1, prices)
sell10 = np.ma.masked_where(np.array(actions) != 4, prices)
buy50 = np.ma.masked_where(np.array(actions) != 2, prices)
sell50 = np.ma.masked_where(np.array(actions) != 5, prices)
buy100 = np.ma.masked_where(np.array(actions) != 3, prices)
hold = np.ma.masked_where(np.array(actions) != 0, prices)
plt.plot(sell10, c = 'yellow')

plt.plot(buy10, c = 'turquoise')
plt.plot(buy50, c = 'lime')
plt.plot(buy100, c = 'green')
plt.plot(sell50, c = 'orange')
plt.plot(sell100, c = 'red')
plt.plot(hold, c = 'blue')

In [ ]:
actions

In [ ]:
plt.plot(assets)

In [ ]:
env = Env_ray(env_config = {'last_states_num': None, 'action_mode': 1, 'version': 'long', 'curiosity_reward':0})
obs = env.test()

done = False
cumulative_reward = 0
prices = []
assets = []
actions = []
states = [obs]
rewards = []
hidden = [torch.zeros(512),torch.zeros(512)]
infos = []
for i in range(6 * 24 * 60):
    action, hidden, info = trainer.compute_action(obs, hidden)
    obs, reward, done, results = env.step(action)
    cumulative_reward += reward
    rewards.append(reward)
    actions.append(action)
    assets.append(env.last_assets)
    prices.append(obs[0])
    states.append(obs)
    infos.append(info)
print("Cumulative reward you've received is: {}. Congratulations!".format(cumulative_reward))
print("Asset_Gain {}".format(assets[-1] -assets[0]))

In [ ]:
sell100 = np.ma.masked_where(np.array(actions) != 6, prices)
buy10 = np.ma.masked_where(np.array(actions) != 1, prices)
sell10 = np.ma.masked_where(np.array(actions) != 4, prices)
buy50 = np.ma.masked_where(np.array(actions) != 2, prices)
sell50 = np.ma.masked_where(np.array(actions) != 5, prices)
buy100 = np.ma.masked_where(np.array(actions) != 3, prices)
hold = np.ma.masked_where(np.array(actions) != 0, prices)

plt.plot(buy10, c = 'turquoise')
plt.plot(buy50, c = 'lime')
plt.plot(buy100, c = 'green')
plt.plot(sell10, c = 'yellow')
plt.plot(sell50, c = 'orange')
plt.plot(sell100, c = 'red')
plt.plot(hold, c = 'blue')

In [ ]:
actions

In [ ]:
plt.plot(assets)

In [ ]:
training['hist_stats/episode_reward'].values[-1]